
# Исследование надежности заемщиков.

**Цель исследования:** проверить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

**Ход исследования**
Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
1. Обзор данных 
2. Предобработка данных 
3. Проверка гипотез


### Шаг 1. Обзор данных

In [315]:
import pandas as pd

In [316]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [317]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Итак, в таблице 12 столбцов. 

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита




Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

### Шаг 2.1 Заполнение пропусков

Сначала посчитаем сколько в таблице пропущенных значений

In [318]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски обнаружены в двух столбцах days_employed и total_income. Оба столбца имеют тип float и являются количественными.
Что бы понять причину пропусков и насколько они могут повлиять, необходимо изучить строки с пропущенными данными и вычислить долю пропусков в датасете.

In [319]:
data[data['days_employed'].isna()].head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [320]:
data[data['total_income'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [321]:
data[data['total_income'].isna()].tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21415,0,NaN,54,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции с жильем
21423,0,NaN,63,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,сделка с автомобилем
21426,0,NaN,49,среднее,1,женат / замужем,0,F,сотрудник,1,NaN,недвижимость
21432,1,NaN,38,неоконченное высшее,2,Не женат / не замужем,4,F,сотрудник,0,NaN,операции с жильем
21463,1,NaN,35,высшее,0,гражданский брак,1,M,сотрудник,0,NaN,на проведение свадьбы
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


In [322]:
#вычислим долю пропусков во всем датсете
part_na = data['total_income'].isna().sum()/len(data['total_income'])
print(part_na)

0.10099883855981417


Доля пропусков составляет 10%. Замечена взаимосвязь пропусков, если есть пропуск в столбце со стажом, то в этой же строке и пропущен ежемесячный доход. Большинство количество пропусков логично, мы можем увидеть пропуски у людей с типом занятости: пенсионер, которые на данный момент не работают, у госслужающих - их доход обычно коммерческая тайна и у сотрудников, на них есть данные во внутренней системе. Что бы не потерять данные и продолжить работу, необходимо заменить пропущенные данные характерными значениями. Так как доход и стаж могут иметь большой разброс, лучшего всего заполнить пропущенные значения медианной.

In [323]:
# отсортируем таблицу по days_employed по возрастанию 
data.sort_values(by = 'days_employed')
# заменим пропущенные значения days_employed медианой
data['days_employed'] = data['days_employed'].fillna(value=data['days_employed'].median())
# проверим количество пропусков в строке
data['days_employed'].isna().sum()

0

In [324]:
# отсортируем таблицу по total_income по возрастанию 
data.sort_values(by = 'total_income')
# заменим пропущенные значения total_income медианой
data['total_income'] = data['total_income'].fillna(value=data['total_income'].median())
# проверим количество пропусков в строке
data['total_income'].isna().sum()

0

### Шаг 2.2 Проверка данных на аномалии и исправления.

Для поиска аномалий необходимо пройтись по столбцам, самые оптимальные методы для поиска аномальных значений: это просмотр значений, выявление минимум или максимума. 

In [325]:
# посмотрим уникальные значения по столбцу children 
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце найдены 2 аномалии: -1 и 20 детей, предлагаю в этих строках заменить медианной значений.

In [326]:
# отсортируем таблицу по children по возрастанию 
data = data.sort_values(by = 'children')
# заменим аномальные значения children медианой
old_value = [-1, 20]
new_value = data['children'].median()
for element in old_value:
    data['children'] = data['children'].replace(element, new_value)

### Комментарий для ревьюера v_1
Это лучше сделать как каждый отдельный элемент заменить или через функцию или другим способом?

In [327]:
# проверим теперь уникальные значения
data['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [328]:
#теперь проверим аномалии столбца days_employed

data['days_employed'].unique()


array([-3.61422023e+03, -2.71041990e+03,  3.70215476e+05, ...,
       -1.33690801e+02, -2.53054052e+03, -3.43804411e+03])

In [329]:
data['days_employed'].min()

-18388.949900568383

In [330]:
data['days_employed'].max()

401755.40047533

Много значений со знаком минус и также в максимальном значении очень болшой стаж. Предпологаю, что произошла ошибка и необходимо взять модуль от значений, так как числа в действительности похожи на реальный стаж просто со знаком минус. а далее проверить сколько значений с невозможно высоким стажем.

In [331]:
# заменим отрицательные значения на положительные
for index in data:
    try:
        data['days_employed'] = data['days_employed'].abs()
    except:
        print(data[index])
        

In [332]:
# проверим замену
data = data.sort_values(by = 'days_employed')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17437,1,24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование
8336,0,24.240695,32,высшее,0,Не женат / не замужем,4,M,сотрудник,0,124115.373655,получение дополнительного образования
6157,2,30.195337,47,среднее,1,гражданский брак,1,M,компаньон,0,231461.185606,свадьба
9683,0,33.520665,43,среднее,1,Не женат / не замужем,4,M,сотрудник,1,128555.897209,приобретение автомобиля
2127,1,34.701045,31,высшее,0,женат / замужем,0,F,компаньон,0,90557.994311,получение образования
5287,1,37.726602,36,среднее,1,женат / замужем,0,F,сотрудник,0,64954.565099,операции со своей недвижимостью
17270,0,39.954170,34,высшее,0,женат / замужем,0,F,госслужащий,0,97264.767002,жилье
13846,2,46.952793,33,среднее,1,женат / замужем,0,F,сотрудник,1,177554.195351,покупка жилья для сдачи
7964,0,47.109840,49,высшее,0,женат / замужем,0,F,сотрудник,0,197545.271278,на покупку подержанного автомобиля
3235,0,50.128298,43,среднее,1,гражданский брак,1,F,компаньон,0,99381.947946,на проведение свадьбы


In [333]:
# теперь посмотрим количество аномально больших значений
count = 0 
for element in data['days_employed']:
    if element > 18000:
        count +=1
print(count)

3446


In [334]:
data.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8369,0,401590.452231,58,среднее,1,женат / замужем,0,F,пенсионер,0,175306.312902,образование
10991,0,401591.828457,56,среднее,1,в разводе,3,F,пенсионер,0,39513.517543,получение дополнительного образования
17823,0,401614.475622,59,среднее,1,женат / замужем,0,F,пенсионер,0,152769.694536,покупка жилья для сдачи
13420,0,401619.633298,63,Среднее,1,гражданский брак,1,F,пенсионер,0,51449.788325,сыграть свадьбу
4697,0,401635.032697,56,среднее,1,женат / замужем,0,F,пенсионер,0,48242.322502,покупка недвижимости
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью


Аномалии очень большого стажа занимают значительную часть данных и встречаются в основном у пенсионеров. Предлагаю заменить данные значения медианной. 

In [335]:
data = data.sort_values(by = 'days_employed')
med_days = data['days_employed'].median()
old_value = []
for element in data['days_employed']:
    if element > 18000:
        old_value.append(element)
new_value = data['days_employed'].median()
for element in old_value:
    data['days_employed'] = data['days_employed'].replace(element, new_value)

In [336]:
data['days_employed'].value_counts()

1808.053434    3447
1203.369529    2175
3231.722690       1
1645.169363       1
1038.987541       1
               ... 
372.393123        1
1124.619628       1
1443.333741       1
4746.367281       1
582.538413        1
Name: days_employed, Length: 15905, dtype: int64

In [337]:
# посмотрим уникальные значения по столбцу dob_years
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

0 лет явно неверное значение, как и многие количественные значения, их лучше всего заменить медианной.

In [338]:
data = data.sort_values(by = 'dob_years')
med_years = data['dob_years'].median()
old_value = [0]
new_value = data['dob_years'].median()
for element in old_value:
    data['dob_years'] = data['dob_years'].replace(element, new_value)

In [339]:
# проверим получилось ли заменить значение
data['dob_years'].value_counts()

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [340]:
# узнаем мин и мкс столбца total_income
data['total_income'].min()

20667.26379327158

In [341]:
data['total_income'].max()

2265604.028722744

Такой разброс в доходах вполне может быть. Предлагаю прейти к следующему шагу.

### Шаг 2.3. Изменение типов данных.

У нас есть два столбца с вещественным типом данных, для удобства расчетов и наглядности предлагаю заменить их на цельночисленный.

In [342]:
try: 
    data['days_employed'] = data['days_employed'].astype('int')
except: 
    print(data)

In [343]:
try:
    data['total_income'] = data['total_income'].astype('int')
except:
    print(data)    

In [344]:
# проверим, вызвав метод info
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 7344 to 8880
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Шаг 2.4. Удаление дубликатов.

Приступим к поиску и устранению явных дубликатов

In [345]:
# посмотрим количество дубликатов в датафрейме 
data.duplicated().sum()

55

In [346]:
# удалим дубликаты, сохранив индексацию
data = data.drop_duplicates().reset_index(drop=True)

In [347]:
# просмотрим уникальные значения столбца education
data['education'].unique()

array(['среднее', 'высшее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ', 'Высшее',
       'неоконченное высшее', 'Неоконченное высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'начальное', 'Начальное', 'НАЧАЛЬНОЕ',
       'ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень'], dtype=object)

В столбце необходимо привести все к одному регистру 

In [348]:
data['education'] = data['education'].str.lower()


In [349]:
#проверим столбец education 
data['education'].unique()

array(['среднее', 'высшее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [350]:
# просмотрим уникальные значения столбца income_type
data['income_type'].unique()

array(['сотрудник', 'госслужащий', 'компаньон', 'пенсионер', 'студент',
       'предприниматель', 'безработный', 'в декрете'], dtype=object)

Во время работы с дубликатами мы обнаружили и убрали явные дубликаты. А так же неявные: в столбце с образованием привели все к нижнему регистру.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два датафрейма, как словари для обращения к ним по id

In [351]:
education_log = data[['education_id', 'education']]

In [352]:
family_log = data[['family_status_id', 'family_status']]

In [353]:
# очистим таблицу education_log сразу же от дубликатов, сохранив индексацию и выведем первые 10 строк
education_log = education_log.drop_duplicates().reset_index(drop=True)
education_log.head(10)

,education_id,education
0,1,среднее
1,0,высшее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [354]:
# очистим таблицу family_log сразу же от дубликатов, сохранив индексацию и выведем первые 10 строк
family_log = family_log.drop_duplicates().reset_index(drop=True)
family_log.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,3,в разводе
2,1,гражданский брак
3,4,Не женат / не замужем
4,2,вдовец / вдова


Теперь удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id и будем обращаться к словворям family_log и education_log по id.

In [355]:
data = data.drop(columns = ['education','family_status'], axis=1)
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,0,401,42,1,0,M,сотрудник,0,158913,операции с жильем
1,2,8035,42,0,3,F,госслужащий,0,112661,покупка жилой недвижимости
2,0,934,42,1,0,F,компаньон,0,201852,покупка недвижимости
3,0,1808,42,1,0,F,пенсионер,0,71291,автомобиль
4,0,6683,42,1,0,F,сотрудник,0,80315,автомобиль
5,0,2497,42,1,0,F,сотрудник,1,91367,недвижимость
6,0,1203,42,1,0,F,сотрудник,0,145017,жилье
7,0,334,42,1,1,M,сотрудник,0,174105,сыграть свадьбу
8,2,1371,42,1,0,M,сотрудник,0,179138,покупка жилья для семьи
9,0,1095,42,0,0,M,сотрудник,0,250757,покупка жилья


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями:
o	0–30000 — 'E';
o	30001–50000 — 'D';
o	50001–200000 — 'C';
o	200001–1000000 — 'B';
o	1000001 и выше — 'A'.



In [356]:
# создадим функцию, которая будет возвращать категорию, в зависимости от дохода. 
def income_group(income):
    if income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A'
  

In [357]:
# создадим отдельный столбец с категориями доходов, и в его ячейках запишем значения, возвращаемые функцией.

data['income_group'] = data['total_income'].apply(income_group)
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_group
0,0,401,42,1,0,M,сотрудник,0,158913,операции с жильем,C
1,2,8035,42,0,3,F,госслужащий,0,112661,покупка жилой недвижимости,C
2,0,934,42,1,0,F,компаньон,0,201852,покупка недвижимости,B
3,0,1808,42,1,0,F,пенсионер,0,71291,автомобиль,C
4,0,6683,42,1,0,F,сотрудник,0,80315,автомобиль,C
5,0,2497,42,1,0,F,сотрудник,1,91367,недвижимость,C
6,0,1203,42,1,0,F,сотрудник,0,145017,жилье,C
7,0,334,42,1,1,M,сотрудник,0,174105,сыграть свадьбу,C
8,2,1371,42,1,0,M,сотрудник,0,179138,покупка жилья для семьи,C
9,0,1095,42,0,0,M,сотрудник,0,250757,покупка жилья,B


### Шаг 2.7. Категоризация целей кредита.

Комментарий для ревьюера: в связи с тем, что в предобработке данных:удаление дубликатов, было не так много уникальных значений и я заменила похожими категориями, считаю нет необходимости в категоризации, так как сейчас в уникальных значениях : array([на недвижимость', 'на образование', 'на свадьбу','покупка автомобиля']. Если данных было больше, то можно как и в категоризации доходов использовать функцию и в условиях использовать 'in'.

In [358]:
# создадим функцию, которая будет возвращать категорию, в зависимости от цели кредита. 
def purpose_group(purpose):
    if 'авто' in purpose:
        return 'покупка автомобиля'
    if 'образован' in purpose:
        return 'на образование'
    if 'свадьб' in purpose:
        return 'на свадьбу'
    return 'на недвижимость'

In [359]:
# создадим отдельный столбец с категориями доходов, и в его ячейках запишем значения, возвращаемые функцией.

data['purpose_group'] = data['purpose'].apply(purpose_group)
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_group,purpose_group
0,0,401,42,1,0,M,сотрудник,0,158913,операции с жильем,C,на недвижимость
1,2,8035,42,0,3,F,госслужащий,0,112661,покупка жилой недвижимости,C,на недвижимость
2,0,934,42,1,0,F,компаньон,0,201852,покупка недвижимости,B,на недвижимость
3,0,1808,42,1,0,F,пенсионер,0,71291,автомобиль,C,покупка автомобиля
4,0,6683,42,1,0,F,сотрудник,0,80315,автомобиль,C,покупка автомобиля
5,0,2497,42,1,0,F,сотрудник,1,91367,недвижимость,C,на недвижимость
6,0,1203,42,1,0,F,сотрудник,0,145017,жилье,C,на недвижимость
7,0,334,42,1,1,M,сотрудник,0,174105,сыграть свадьбу,C,на свадьбу
8,2,1371,42,1,0,M,сотрудник,0,179138,покупка жилья для семьи,C,на недвижимость
9,0,1095,42,0,0,M,сотрудник,0,250757,покупка жилья,B,на недвижимость


В предобработке данных, мы удалили явные и неявные дубликаты, нашли и устранили пропуски, с помощью замен. поработали с аномальным значениями в каждом столбце, изменили тип данных для возможности совершения операций с ними. И во избежании ошибок, создали словари для удобства обращения к ним и добавили дополнительные столбцы с категориями, для удобства обращения к данным и их анализу. Теперь данные не содержат пропусков, повторений и удобны для анализа.

### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [360]:
#построим сводную таблицу с количеством должников и их отношению к общему количеству заемщиков по количеству детей
print(data.pivot_table(index='children', values='debt', aggfunc = (['count', 'sum', 'mean'])))

          count   sum      mean
           debt  debt      debt
children                       
0         14229  1072  0.075339
1          4809   444  0.092327
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000


##### Вывод 1:

На основании вышеупомянутых данных, можно увидеть, что совсем нет должников у кого 5-ть детей, но скорей всего это из-за того что выборка не репрезентативна, так как таких людей очень мало в данных. Так же, самый маленький процент у кого детей нет. Все остальные примерно на одном уровне. Поэтому здесь скорей всего играет роль есть ли дети или нет. 

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [361]:
#построим сводную таблицу с количеством должников и их отношению к общему количеству заемщиков по семейному статусу
data_group = data.pivot_table(index='family_status_id', values='debt', aggfunc = (['count', 'sum', 'mean']))

In [362]:
# для наглядности объединим две таблицы
data_family = family_log.merge(data_group, on='family_status_id', how='left')
data_family


/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,family_status_id,family_status,"(count, debt)","(sum, debt)","(mean, debt)"
0,0,женат / замужем,12344,931,0.075421
1,3,в разводе,1195,85,0.071130
2,1,гражданский брак,4162,388,0.093224
3,4,Не женат / не замужем,2810,274,0.097509
4,2,вдовец / вдова,959,63,0.065693


##### Вывод 2:

Больше всего имеют долги те, кто находится в гражданском браке или не женат/не замужем. Меньше всего имеют задолжность вдовцы - но сам процент вдовцов небольшой, поэтому выборка по ним не совсем репрезентативна.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [363]:
#построим сводную таблицу с количеством должников и их отношению к общему количеству заемщиков по доходу
print(data.pivot_table(index='income_group', values='debt', aggfunc = (['count', 'sum', 'mean'])))

              count   sum      mean
               debt  debt      debt
income_group                       
A                25     2  0.080000
B              5041   356  0.070621
C             16032  1360  0.084830
D               350    21  0.060000
E                22     2  0.090909


Наименьшее количесто должников, на удивление, среди людей с доходами от 30 000 - 50 000 рублей, возможно потому что они берут не такие масштабные кредиты. Построим сводную таблицу, что б проверить это.

In [364]:
# сводная таблица по количеству кредитов на каждую категорию
data_pivot = data.pivot_table(index='income_group', columns='purpose_group', values='debt', aggfunc ='count')
data_pivot = data_pivot.reset_index()                                     
data_pivot.head(10)

purpose_group,income_group,на недвижимость,на образование,на свадьбу,покупка автомобиля
0,A,17,4,2,2
1,B,2568,897,545,1031
2,C,8050,3032,1749,3201
3,D,168,76,34,72
4,E,11,5,4,2


In [365]:
#сводная таблица по количеству долгов на каждую категорию
data_pivot = data.pivot_table(index='income_group', columns='purpose_group', values='debt', aggfunc ='sum')
data_pivot = data_pivot.reset_index()                                     
data_pivot.head(10)

purpose_group,income_group,на недвижимость,на образование,на свадьбу,покупка автомобиля
0,A,1,1,0,0
1,B,172,70,28,86
2,C,601,297,155,307
3,D,6,2,3,10
4,E,2,0,0,0


##### Вывод 3:

Наименьшее количесто должников, cреди людей с доходами от 30 000 - 50 000 рублей и берут они больше всего кредитов на недвижимость, поэтому здесь скорей всего, зависимость от цели кредита, а не от дохода.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [366]:
#построим сводную таблицу с количеством должников и их отношению к общему количеству заемщиков по доходу
print(data.pivot_table(index='purpose_group', values='debt', aggfunc = (['count', 'sum', 'mean'])))

                    count  sum      mean
                     debt debt      debt
purpose_group                           
на недвижимость     10814  782  0.072314
на образование       4014  370  0.092177
на свадьбу           2334  186  0.079692
покупка автомобиля   4308  403  0.093547


##### Вывод 4:

Меньше должников среди тех кто берет кредиты на свадьбу или на недвижимость. Возможно это связано с тем, что свадьба это чаще всего не очень большой по сумме кредит, а недвижимость чаще всего остается в залоге у банка и что бы не потерять жилье, люди платят более исправно.

## Общий вывод:

На входе мы получили датасет с 21000 строк с информацией о заемщиках, в ходе предобработки данных, мы исправились от пропусков, дубликатов, создали более наглядную таблицу с категориями, словарями и без пропусков. 
В ходе анализа с помощью сводных таблиц мы ответили на 4 вопроса и установили какие факторы влияеют на возврат кредита в срок: 
1. Количество детей не влияет на возврат кредита во время, больше влияет факт наличия или отсутствия детей
2. Больше всего задолжностей среди тех людей, кто состоит в гражданском браке или не женат/не замужем
3. Доход, на удивление, не влияет на возвра кредита в срок.
4. Чаще всего во время возращают кредит те, кто берет его на недвижимость или свадьбу.

